In [1]:
import os
import warnings
import requests
import statistics
import json
from tqdm import tqdm
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

tqdm.pandas()
warnings.filterwarnings("ignore")

In [2]:
output_path = "./student_grades.json"
students_path = "./major_minor/students.csv"

force_request = 1
session_id = "fmrb5gkzz2q55ilno0nejr4b"

In [3]:
def grab_sysgen(session_id, ashoka_email):
    url = "https://ams.ashoka.edu.in/Contents/Masters/ViewstudentDirectory.aspx/GetStudentList"
    headers = {
        "Host": "ams.ashoka.edu.in",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-GB,en-US;q=0.7,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/json; charset=utf-8",
        "X-Requested-With": "XMLHttpRequest",
        "Content-Length": "260",
        "Origin": "https://ams.ashoka.edu.in",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://ams.ashoka.edu.in/Contents/Masters/ViewstudentDirectory.aspx",
        "Cookie": "ASP.NET_SessionId=" + session_id + "; __AJAXAntiXsrfToken=pqqFQQCNulHpSROVS20HjZGCxmhQksGUTzLu756vcdAjU5gAra9hA_6GHr34sMZPXJwwlNjwhgvJxZ7-xCGziZ8QAVPjiljAXeo1ALMW4T01",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
    }
    post_data = json.dumps(
        {
            "XMLData": "<tbl><tr><ProgrammeSysGenId></ProgrammeSysGenId><SessionSysGenId"
                       "></SessionSysGenId "
                       "><ScheduleSysGenId></ScheduleSysGenId><CourseSysGenId></CourseSysGenId><LSNo></LSNo><DSNo"
                       "></DSNo><SearchByText>" + ashoka_email + "</SearchByText></tr></tbl>",
            "ListType": "All",
        }
    )

    try:
        response = requests.post(url, data=post_data, headers=headers)
        return str(response.content).split('\\\\"UserSysGenId\\\\": \\\\"')[1][0:11]
    except:
        return "0000000000"

In [4]:
def grab_grade(session_id, ashoka_email, sysgen_id, force_request, file_name):
    if not force_request:
        with open(file_name, 'r') as file:
            file_objects = json.load(file)
            for student_index, student_object in enumerate(file_objects["data"]):
                if ashoka_email in student_object["AshokaEmailID"] or ashoka_email in student_object["Name"]:
                    print("Found in file!")
                    return student_object

    print(f"Sending request for ID {sysgen_id}, email {ashoka_email}...")
    url = "https://ams.ashoka.edu.in/Services/MasterServices.asmx/GetCourseReport"

    headers = {
        "Host": "ams.ashoka.edu.in",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-GB,en-US;q=0.7,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/json; charset=utf-8",
        "X-Requested-With": "XMLHttpRequest",
        "Content-Length": "260",
        "Origin": "https://ams.ashoka.edu.in",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://ams.ashoka.edu.in/Contents/CourseManagement/StudentCourseReport_Student.aspx",
        "Cookie": "ASP.NET_SessionId=" + session_id + "; __AJAXAntiXsrfToken=_5B4rluYFBXxsvnk7hMJt1rk8slaVMdzjvFRVxU91q6nO2Bx7tKqUKHVxYUHtyJlDDOh2oktIyVR_xwJBM2H6E8xtWvhNOFoqrn-_1pcVOg",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
    }
    post_data = json.dumps(
        {"xml": "<tbl><tr><UserSysGenId>" + sysgen_id + "</UserSysGenId></tr></tbl>"}
    )

    response = requests.post(url, data=post_data, headers=headers)

    return (
        parse_grade(response.content.replace(b"\\r", b"").replace(b"\\n", b"").replace(b'\\"', b'"'))
    )

In [5]:
def parse_grade(json_string):
    json_data = {}
    json_string = str(json_string)[2:]
    if "ProgrammeSysGenId" not in json_string:
        return None

    json_data["ProgrammeSysGenId"] = json_string.split('ProgrammeSysGenId": "')[1].split('",      "Name"')[0]
    json_data["Name"] = json_string.split('Name": "')[1].split('",      "CurrentCGPA"')[0]
    json_data["CurrentCGPA"] = json_string.split('CurrentCGPA": "')[1].split('",      "MajorCGPA"')[0]
    json_data["MajorCGPA"] = json_string.split('MajorCGPA": "')[1].split('",      "OVERALL_CREDITS"')[0]
    json_data["AshokaID"] = json_string.split('AshokaId": "')[1].split('",      "AshokaEmailId"')[0]
    json_data["AshokaEmailID"] = json_string.split('AshokaEmailId": "')[1].split('",      "ProgrammeSysGenId"')[0]

    semesters = json_string.split('"CourseCode"')[0].split('"Semester":')[1:-1]
    json_data["semesters"] = []
    for semester in semesters:
        semester_json = {}
        semester_json["semester"] = (
            semester.split(' "')[1].split('",      "Status')[0].strip('",     ')
        )
        semester_json["GPA"] = float(
            semester.split('"GPA": "')[1].split('",      "SemesterCGPA')[0]
        )
        semester_json["CGPA"] = float(
            semester.split('"SemesterCGPA": "')[1].split('",      "Deans_List"')[0]
        )
        semester_json["Dean's List"] = (
            True
            if "Dean"
               in semester.split('"Deans_List": "')[1].split(
                '"    },    {      "ScheduleSysGenId'
            )[0]
            else False
        )
        json_data["semesters"].append(semester_json)

    courses = json_string.split('"Table3": [')[1].split("ScheduleSysGenId")[1:]
    json_data["courses"] = []
    for course in courses:
        course_json = {}
        course_json["CourseSysGenId"] = course.split('"CourseSysGenId": "')[1].split(
            '",      "Semester"'
        )[0]
        course_json["semester"] = course.split('"Semester": "')[1].split(
            '",      "CourseCode"'
        )[0]
        course_json["code"] = course.split('"CourseCode": "')[1].split(
            '",      "EquivalentCourseCode"'
        )[0]
        course_json["credits"] = course.split('"Credits": ')[1].split(
            ',      "GradePoint"'
        )[0]
        course_json["name"] = course.split('"CourseName": "')[1].split(
            '",      "Grade'
        )[0]
        course_json["grade"] = course.split('"Grade": "')[1].split(
            '",      "CountinCredits"'
        )[0]
        json_data["courses"].append(course_json)

    return json_data

In [6]:
def update_file(file_name, new_object):
    with open(file_name, 'r') as file:
        file_objects = json.load(file)

    for student_index, student_object in enumerate(file_objects["data"]):

        if new_object["AshokaEmailID"] == student_object["AshokaEmailID"]:
            if student_object == new_object:
                return "Exists in file!"

            else:
                file_objects["data"][student_index] = new_object
                with open(file_name, 'w') as file:
                    json.dump(file_objects, file, indent=4)
                return "Updated in file!"

    else:
        file_objects["data"].append(new_object)
        with open(file_name, 'w') as file:
            json.dump(file_objects, file, indent=4)
        return "Appended to file!"

In [7]:
students = pd.read_csv(students_path)
students

,batch,email,id,name,status,major,minor,program,grad,cgpa,mgpa
0,ASP 2017-18,aania_asp18@ashoka.edu.in,UG-14-1609,Aania,Graduated,NaN,NaN,ASP,2018.0,2.64,0.0
1,ASP 2017-18,aashna.lal_asp18@ashoka.edu.in,UG-14-0568,Aashna Lal,Graduated,NaN,CW,ASP,2018.0,3.36,0.0
2,ASP 2017-18,abhinav.srikant_asp18@ashoka.edu.in,UG-14-1833,Abhinav Srikant,Graduated,PSY,MS,ASP,2018.0,3.14,0.0
3,ASP 2017-18,aditya.prakash_asp18@ashoka.edu.in,UG-14-1812,Aditya Prakash,Graduated,NaN,MAT,ASP,2018.0,3.57,0.0
4,ASP 2017-18,ahan.bezbaroa_asp18@ashoka.edu.in,UG-14-0336,Ahan Bezbaroa,Graduated,PPE,ENG,ASP,2018.0,3.50,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7001,UG 2023-27,yuvakshi.dam_ug2023@ashoka.edu.in,1020231747,Yuvakshi Dam,Enrolled,NaN,NaN,UG,2027.0,NaN,NaN
7002,UG 2023-27,yuvraj.verma_ug2023@ashoka.edu.in,1020231748,Yuvraj Verma,Enrolled,NaN,NaN,UG,2027.0,NaN,NaN
7003,UG 2023-27,zahra.baqeri_ug2023@ashoka.edu.in,1020231020,Zahra Baqeri,Enrolled,NaN,NaN,UG,2027.0,NaN,NaN
7004,UG 2023-27,zahrah.imani_ug2023@ashoka.edu.in,1020231749,Zahrah Salim Imani,Enrolled,NaN,NaN,UG,2027.0,NaN,NaN


In [20]:
cgpa, mgpa = [], []
for student_email in students.loc[:, "email"]:
    try:
        student_sysgen = grab_sysgen(session_id, student_email)
        student_grades = grab_grade(session_id, student_email, student_sysgen, force_request, output_path)

        cgpa.append((student_email, student_grades["CurrentCGPA"]))
        mgpa.append((student_email, student_grades["MajorCGPA"]))
        print(update_file(output_path, student_grades))
    except:
        cgpa.append((student_email, 0))
        mgpa.append((student_email, 0))
        pass

Sending request for ID USR00024312, email heet.dhawale_ug2023@ashoka.edu.in...
Exists in file!
Sending request for ID USR00026970, email hitanshi.shah_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00024508, email hitee.bhardwaj_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00024210, email hitova.assumi_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00024431, email hiyaneijemmy.das_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00026475, email hrishika.popli_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00026661, email hrithika.tom_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00024900, email hudsonmanoj.varughese_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00027015, email imbesat.fatima_ug2023@ashoka.edu.in...
Appended to file!
Sending request for ID USR00026378, email irene.philip_ug2023@ashoka.edu.in...
Appended to file!
Sending req

In [21]:
students = pd.merge(students, pd.DataFrame(cgpa, columns=["email", "cgpa"]), how="left", on=["email"], suffixes=("_og", "_new"))
students = pd.merge(students, pd.DataFrame(mgpa, columns=["email", "mgpa"]), how="left", on=["email"], suffixes=("_og", "_new"))
students["cgpa"] = students.apply(lambda x: x["cgpa_new"] if not pd.isna(x["cgpa_new"]) else x["cgpa_og"] if not pd.isna(x["cgpa_og"]) else None, axis=1)
students["mgpa"] = students.apply(lambda x: x["mgpa_new"] if not pd.isna(x["mgpa_new"]) else x["mgpa_og"] if not pd.isna(x["mgpa_og"]) else None, axis=1)
students.drop(["cgpa_new", "cgpa_og", "mgpa_new", "mgpa_og"], axis=1, inplace=True)
students.to_csv(students_path, index=False)

In [22]:
students

,batch,email,id,name,status,major,minor,program,grad,cgpa,mgpa
0,ASP 2017-18,aania_asp18@ashoka.edu.in,UG-14-1609,Aania,Graduated,NaN,NaN,ASP,2018.0,2.64,0.0
1,ASP 2017-18,aashna.lal_asp18@ashoka.edu.in,UG-14-0568,Aashna Lal,Graduated,NaN,CW,ASP,2018.0,3.36,0.0
2,ASP 2017-18,abhinav.srikant_asp18@ashoka.edu.in,UG-14-1833,Abhinav Srikant,Graduated,PSY,MS,ASP,2018.0,3.14,0.0
3,ASP 2017-18,aditya.prakash_asp18@ashoka.edu.in,UG-14-1812,Aditya Prakash,Graduated,NaN,MAT,ASP,2018.0,3.57,0.0
4,ASP 2017-18,ahan.bezbaroa_asp18@ashoka.edu.in,UG-14-0336,Ahan Bezbaroa,Graduated,PPE,ENG,ASP,2018.0,3.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7001,UG 2023-27,yuvakshi.dam_ug2023@ashoka.edu.in,1020231747,Yuvakshi Dam,Enrolled,NaN,NaN,UG,2027.0,3.22,0.00
7002,UG 2023-27,yuvraj.verma_ug2023@ashoka.edu.in,1020231748,Yuvraj Verma,Enrolled,NaN,NaN,UG,2027.0,3.19,0.00
7003,UG 2023-27,zahra.baqeri_ug2023@ashoka.edu.in,1020231020,Zahra Baqeri,Enrolled,NaN,NaN,UG,2027.0,0.00,0.00
7004,UG 2023-27,zahrah.imani_ug2023@ashoka.edu.in,1020231749,Zahrah Salim Imani,Enrolled,NaN,NaN,UG,2027.0,3.82,0.00
